In [12]:
import requests
from bs4 import BeautifulSoup
import time
import random
import csv
import re

# Definimos un header con el User-Agent de una Mac para evitar baneos
user_agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1'
]

# Función para obtener headers aleatorios
def obtener_headers():
    headers = {
        'User-Agent': random.choice(user_agents)
    }
    return headers

# Función para realizar la solicitud con manejo de reintentos
def realizar_solicitud(url, intentos=0, max_reintentos=5, tiempo_espera=5):
    try:
        response = requests.get(url, headers=obtener_headers())
        
        # Si el código de estado es 429, esperar y reintentar
        if response.status_code == 429:
            if intentos < max_reintentos:
                print(f"[WARNING] Error 429: Too Many Requests. Reintentando en {tiempo_espera} segundos... (Intento {intentos+1}/{max_reintentos})")
                time.sleep(tiempo_espera)
                return realizar_solicitud(url, intentos + 1, max_reintentos, tiempo_espera)
            else:
                print(f"[ERROR] Se superó el límite de reintentos para la URL: {url}")
                return None
        elif response.status_code == 200:
            return response
        else:
            print(f"[ERROR] Código de estado {response.status_code} para la URL: {url}")
            return None
    except Exception as e:
        print(f"[ERROR] Ocurrió un error al hacer la solicitud: {str(e)}")
        return None

# Función para obtener los títulos y los enlaces de las películas en cada página
def obtener_enlaces_peliculas(url):
    response = realizar_solicitud(url)
    if response:
        soup = BeautifulSoup(response.text, 'html.parser')
        movie_links = []
        movie_titles = soup.find_all('div', class_='movie-title')

        if not movie_titles:
            print(f"[DEBUG] No se encontraron películas en la URL: {url}")
            return []

        for title in movie_titles:
            a_tag = title.find('a')
            if a_tag:
                enlace = a_tag.get('href')
                nombre = a_tag.get('title').strip()
                if not enlace.startswith('https'):
                    enlace = f"https://www.filmaffinity.com{enlace}"
                movie_links.append({'name': nombre, 'link': enlace})
        return movie_links
    else:
        return []

# Función para obtener la información de la película con reintentos
def obtener_criticas(url_pelicula):
    response = realizar_solicitud(url_pelicula)
    if response:
        soup = BeautifulSoup(response.text, 'html.parser')
        movie_info = soup.find('dl', class_='movie-info')

        if movie_info:
            original_title = movie_info.find('dt', text='Título original').find_next_sibling('dd').get_text(strip=True) if movie_info.find('dt', text='Título original') else None
            year = movie_info.find('dt', text='Año').find_next_sibling('dd').get_text(strip=True) if movie_info.find('dt', text='Año') else None
            duration = movie_info.find('dt', text='Duración').find_next_sibling('dd').get_text(strip=True) if movie_info.find('dt', text='Duración') else None
            country = movie_info.find('dt', text='País').find_next_sibling('dd').get_text(strip=True) if movie_info.find('dt', text='País') else None
            genre = movie_info.find('dt', text='Género').find_next_sibling('dd').get_text(strip=True) if movie_info.find('dt', text='Género') else None

            review_container = soup.find('div', id='review-container')
            if review_container:
                reviews_box = review_container.find('div', id='movie-reviews-box')
                if reviews_box:
                    a_tag = reviews_box.find('a', text=lambda t: 'críticas' in t.lower())
                    if a_tag:
                        href_reviews = a_tag.get('href')
                        review_count = reviews_box.get_text(strip=True).split()[0]

                        if not href_reviews.startswith('https'):
                            href_reviews = f"https://www.filmaffinity.com{href_reviews}"

                        return {
                            'original_title': original_title,
                            'year': year,
                            'duration': duration,
                            'country': country,
                            'genre': genre,
                            'reviews_link': href_reviews,
                            'reviews_count': review_count
                        }

            return {
                'original_title': original_title,
                'year': year,
                'duration': duration,
                'country': country,
                'genre': genre,
                'reviews_link': None,
                'reviews_count': '0'
            }
        else:
            print(f"[WARNING] No se encontró 'movie-info' en la página de la película: {url_pelicula}")
            return None
    else:
        return None

# Función para limpiar el texto de reviews_count
def limpiar_reviews_count(review_count):
    match = re.search(r'\d+', review_count)
    return match.group(0) if match else '0'

# Bucle para iterar sobre la paginación de las películas
pagina = 1
csv_filename = 'movies_and_reviews.csv'

with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Movie', 'Movie Link', 'Year', 'Duration', 'Country', 'Genre', 'Reviews Link', 'Number of Reviews', 'User', 'Score', 'Review Text'])

while True:
    try:
        url = f"https://www.filmaffinity.com/es/category.php?id=new_netflix&page={pagina}"
        print(f"[INFO] Scrapeando la página {pagina}...")

        enlaces_peliculas = obtener_enlaces_peliculas(url)
        if not enlaces_peliculas:
            print("[INFO] No hay más películas.")
            break

        for pelicula in enlaces_peliculas:
            print(f"[INFO] Procesando película: {pelicula['name']} - Enlace: {pelicula['link']}")

            criticas_info = obtener_criticas(pelicula['link'])
            if criticas_info:
                pelicula.update(criticas_info)
                pelicula['reviews_count'] = limpiar_reviews_count(pelicula['reviews_count'])

                detalles_criticas = []
                if criticas_info['reviews_link']:
                    detalles_criticas = obtener_detalles_criticas(criticas_info['reviews_link'])

                with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    if detalles_criticas:
                        for critica in detalles_criticas:
                            writer.writerow([
                                pelicula['name'],
                                pelicula['link'],
                                pelicula['year'],
                                pelicula['duration'],
                                pelicula['country'],
                                pelicula['genre'],
                                pelicula['reviews_link'],
                                pelicula['reviews_count'],
                                critica['user'],
                                critica['score'],
                                critica['text']
                            ])
                    else:
                        writer.writerow([
                            pelicula['name'],
                            pelicula['link'],
                            pelicula['year'],
                            pelicula['duration'],
                            pelicula['country'],
                            pelicula['genre'],
                            pelicula['reviews_link'],
                            pelicula['reviews_count'],
                            None, None, None
                        ])

        pagina += 1
        time.sleep(0.5)

    except Exception as e:
        print(f"[ERROR] Ocurrió un error en el bucle principal: {str(e)}")
        break

print(f"[INFO] Proceso completado. Datos guardados en {csv_filename}")


[INFO] Scrapeando la página 1...
[WARNING] Error 429: Too Many Requests. Reintentando en 5 segundos... (Intento 1/5)


KeyboardInterrupt: 